# settings

In [17]:
import os
import sys
import time
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import re
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import json
import pickle
from sklearn.model_selection import train_test_split
%matplotlib inline
# nltk.download()
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer

import math
import os
import random
import string

# Pytorch library for training
import torch
from torch import optim

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


# from torchtext.data import Field, BucketIterator, Example

#from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# pre-trained word embedding by GolVe
https://github.com/stanfordnlp/GloVe
...Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 300d vectors, 822 MB download): glove.6B.zip
Due to the nature of WikiHow dataset, we choose word embedding result file 'glove.6B' which is pre-trained on Wikipedia and Gigaword dataset. Besides, it contains four .text for different embdedding vector length: 50, 100, 200, 300. 

In [18]:
# # Choose embedding dimention
# embed_dim = 200

# # Download and unzip GloVe embedding
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

--2021-05-30 13:15:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-05-30 13:15:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
Unable to establish SSL connection.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# # input your pre-train txt path and parse the data
# path = './glove.6B/glove.6B.200d.txt'

# embed_dict = {}
# with open(path,'r') as f:
#     lines = f.readlines()
#     for l in lines:
#         w = l.split()[0]
#         v = np.array(l.split()[1:]).astype('float')
#         embed_dict[w] = v

# embed_dict['@@_unknown_@@'] = np.random.random(embed_dim)

# # remove all the unnecesary files
# !rm -rf glove.6B.zip
# !rm -rf glove.6B.50d.txt
# !rm -rf glove.6B.100d.txt
# !rm -rf glove.6B.200d.txt
# !rm -rf glove.6B.300d.txt

# # check the length of the dictionary
# len(embed_dict.keys())

## Load WikiHow Dataset
Download dataset from https://ucsb.app.box.com/s/7yq601ijl1lzvlfu4rjdbbxforzd2oag

In [19]:
%%time
data = pd.read_csv('.\Dataset\wikihowSep.csv')
data = data[:1000]
data = data.astype(str)
rows, columns = data.shape

Wall time: 35.7 s


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

expand contraction by https://github.com/khurram6968/NLP-Expand-Contraction-Python/blob/master/NLP.py

In [22]:
contraction_map={
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd've": "how did have",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "might have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "shall'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "will't've": "will not have",
    "would've": "would have",
    "would't": "would not",
    "would't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you have all",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
}

def expand_contractions(sent, mapping):
    #pattern for matching contraction with their expansions
    pattern = re.compile('({})'.format('|'.join(mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    
    def expand_map(contraction):
        #using group method to access subgroups of the match
        match = contraction.group(0)
        #to retain correct case of the word
        first_char = match[0]
        #find out the expansion
        expansion = mapping.get(match) if mapping.get(match) else mapping.get(match.lower())
        expansion = first_char + expansion[1:]
        return expansion
    #using sub method to replace all contractions with their expansions for a sentence
    #function expand_map will be called for every non overlapping occurence of the pattern
    expand_sent = pattern.sub(expand_map, sent)
    return expand_sent

In [24]:
def data_loader(dataframe, target_col): 
    # Extraction from dataframe into a list
    text = [article for article in getattr(dataframe, target_col)]
    
    # Removing accented characters
    text = [unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf-8', 'ignore') for sentence in text]
    
    # Expanding contractions
    text = [expand_contractions(sentence, contraction_map) for sentence in text]

    # Removing special characters
    pat1 = r'[^a-zA-z0-9.,!?\s]' 
    # pat1 = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = [re.sub(pat1, '', sentence) for sentence in text]
    
    # Removing extra commas
    pat2 = r'[.]+[\n]+[,]'
    text = [re.sub(pat2,".\n", sentence) for sentence in text]
    
    # Removing extra whitespaces and tabs
    # pat3 = r'^\s*|\s\s*'
    pat3 = r'^\s+$|\s+$'
    text = [re.sub(pat3, '', sentence).strip() for sentence in text]
    
    # Add space before '.'
    pat4 = r'\.|\?|\！|\,'
    text = [re.sub(pat4, ' ', sentence) for sentence in text]
    
    # Lowercase
    text = [sentence.lower() for sentence in text]
    
    # Tokenize
    text = [('sos ' + sentence + ' eos').split() for sentence in text]
    
    return np.array(text, dtype=object)

In [32]:
%%time
text_data = data_loader(data, 'text')
headline_data = data_loader(data, 'headline')

Wall time: 1.98 s


#### Remove text and headline that are too long or too short

In [33]:
data['num_word'] = [len(text_data[i]) for i in range(text_data.shape[0])]
num_word = np.sort(data['num_word'])

data['num_word_hl'] = [len(headline_data[i]) for i in range(headline_data.shape[0])]
num_word_hl = np.sort(data['num_word_hl'])

min_text_len = num_word[int(len(num_word)*0.1)]
max_text_len = num_word[int(len(num_word)*0.95)]

min_hl_len = num_word_hl[int(len(num_word_hl)*0.1)]

In [34]:
headline_ratio_threshold = 0.75

del_idx = []
for i in range(rows):
    # Remove too short text
    if data['num_word'][i] < min_text_len:
        del_idx.append(i)
    else:
        pass
    
    # Remove too long text
    if max_text_len < data['num_word'][i]:
        del_idx.append(i)
    else:
        pass
    
    # Remove headline longer than 0.75*len(text)
    if data['num_word_hl'][i] > headline_ratio_threshold*data['num_word'][i]:
        del_idx.append(i)
    else:
        pass
    
    # Remove too short headline
    if data['num_word_hl'][i] < min_hl_len:
        del_idx.append(i)
    else:
        pass
    
    
# Number of removed index
print(len(np.unique(del_idx)))


246


In [35]:
text_data = np.delete(text_data, del_idx, axis=0)
headline_data = np.delete(headline_data, del_idx, axis=0)

In [36]:
text_data.shape[0], headline_data.shape[0]

(754, 754)

#### Split train, test, validation datasets

In [37]:
text_train, text_test, headline_train, headline_test = train_test_split(text_data, headline_data, test_size=0.1, random_state=1)

text_train, text_dev, headline_train, headline_dev = train_test_split(text_train, headline_train, test_size=0.1, random_state=1)

#### Sort sentence from longer to shorter length

In [38]:
def data_sorter(text, headline): 
    headline = [y for x,y in sorted(zip(text, headline), key = lambda pair: len(pair[0]), reverse = True)]
    text = list(text)
    text.sort(key = lambda x: len(x), reverse = True)

    return np.array(text), np.array(headline)

In [39]:
text_train, headline_train = data_sorter(text_train, headline_train)
text_test,  headline_test  = data_sorter(text_test, headline_test)
text_dev,   headline_dev   = data_sorter(text_dev, headline_dev)

<ipython-input-38-515ae7e24249>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(text), np.array(headline)


### Save dataset

In [40]:
# train
np.save('./Dataset3/text_train.npy', text_train)
np.save('./Dataset3/headline_train.npy', headline_train)

# dev
np.save('./Dataset3/text_dev.npy', text_dev)
np.save('./Dataset3/headline_dev.npy', headline_dev)

# test
np.save('./Dataset3/text_test.npy', text_test)
np.save('./Dataset3/headline_test.npy', headline_test)

## Build Vocabulary
https://www.kdnuggets.com/2019/11/create-vocabulary-nlp-tasks-python.html

In [41]:
class Vocabulary:
    PAD_token = 0   # Used for padding short sentences
    SOS_token = 1   # Start-of-sentence token
    EOS_token = 2   # End-of-sentence token
    UNK_token = 3   # Out-of-vocabulary token

    def __init__(self, name):
        self.name = name
        self.word2index = {"pad":0, "sos":1, "eos":2, "unk":3}
        self.word2count = {"pad":0, "sos":0, "eos":0, "unk":0}              
        self.index2word = {0: "pad", 1: "sos", 2: "eos", 3: "unk"}
        self.num_words = 4
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in sentence:           
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            self.longest_sentence = sentence_len
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

In [42]:
text_vocabulary = Vocabulary('text')
headline_vocabulary = Vocabulary('headline')

#### Generate text and headline vocabulary

In [43]:
# Generate text vocabulary
for i in range(len(text_train)):
    text_vocabulary.add_sentence(text_train[i])
    for word in text_train[i]:
        text_vocabulary.add_word(word)
text_train_vocab_len = len(text_vocabulary.word2index.keys())
    
# Index test and dev dataset using text_vocabulary
for i in range(len(text_dev)):
    for word in text_dev[i]:
        text_vocabulary.add_word(word)
for i in range(len(text_test)):
    for word in text_test[i]:
        text_vocabulary.add_word(word)

In [44]:
# Generate headline vocabulary
for i in range(len(headline_train)):
    headline_vocabulary.add_sentence(headline_train[i])
    for word in headline_train[i]:
        headline_vocabulary.add_word(word)
hl_train_vocab_len = len(headline_vocabulary.word2index.keys())
    
    
# Index test and dev dataset using headline_vocabulary
for i in range(len(headline_dev)):
    for word in headline_dev[i]:
        headline_vocabulary.add_word(word)
for i in range(len(headline_test)):
    for word in headline_test[i]:
        headline_vocabulary.add_word(word)

In [45]:
print(text_train_vocab_len)
print(hl_train_vocab_len)

print(len(text_vocabulary.word2index.keys()))
print(len(headline_vocabulary.word2index.keys()))

6492
1396
7239
1593


#### Word to index and padding

In [46]:
def w2in(data,vocabulary):
    data_idx = []
    lengths = []
    for i in range(len(data)):
        idx = []
        lengths.append(len(data[i]))
        for word in data[i]:
            idx.append(vocabulary.to_index(word))
        data_idx.append(torch.tensor(idx))
    lengths = torch.tensor(lengths)
    data_pad = torch.nn.utils.rnn.pad_sequence(data_idx, batch_first=True, padding_value=0.0)
    return data_idx, data_pad, lengths


In [47]:
def batchpad_data(data):
    voc = Vocabulary('text')
    data_idx = []
    for i in range(len(data)):
        idx = []
        for word in data[i]:
            voc.add_word(word)
        for word in data[i]:
            idx.append(voc.to_index(word))
        data_idx.append(torch.tensor(idx))
    data_pad = torch.nn.utils.rnn.pad_sequence(data_idx, batch_first=True, padding_value=0.0)
    return data_idx, data_pad, voc

In [48]:
# Compute data_idx, data_pad and data_lengths
text_train_idx, text_train_pad, text_train_lengths = w2in(text_train,text_vocabulary)
headline_train_idx, headline_train_pad, headline_train_lengths = w2in(headline_train,headline_vocabulary)

text_dev_idx, text_dev_pad, text_dev_lengths = w2in(text_dev,text_vocabulary)
headline_dev_idx, headline_dev_pad, headline_dev_lengths = w2in(headline_dev, headline_vocabulary)

text_test_idx, text_test_pad, text_test_lengths = w2in(text_test,text_vocabulary)
headline_test_idx, headline_test_pad, headline_test_lengths = w2in(headline_test, headline_vocabulary)

#### Pad mask

In [49]:
# Compute pad_mask of each dataset (position with padding = 1, no padding = 0)

# Training data pad_mask
text_train_padmask = torch.where(text_train_pad == 0, 1, 0)
headline_train_padmask = torch.where(headline_train_pad == 0, 1, 0)

# Dev data pad_mask
text_dev_padmask = torch.where(text_dev_pad == 0, 1, 0)
headline_dev_padmask = torch.where(headline_dev_pad == 0, 1, 0)

# Testing data pad_mask
text_test_padmask = torch.where(text_test_pad == 0, 1, 0)
headline_test_padmask = torch.where(headline_test_pad == 0, 1, 0)

In [50]:
# print(text_train[-1])
# print(text_test_pad[-1])

#### Out of vocabulary lengths

In [51]:
# number of OOV in each sentence in the train set: suppose to be 0 for every sentence
text_train_oov = torch.sum(torch.where(text_train_pad > text_train_vocab_len, 1, 0), axis = 1)
headline_train_oov = torch.sum(torch.where(headline_train_pad > hl_train_vocab_len, 1, 0), axis = 1)

# print(text_train_oov, headline_train_oov)

# number of OOV in each sentence in the dev set
text_dev_oov = torch.sum(torch.where(text_dev_pad > text_train_vocab_len, 1, 0), axis = 1)
headline_dev_oov = torch.sum(torch.where(headline_dev_pad > hl_train_vocab_len, 1, 0), axis = 1)

# number of OOV in each sentence in the test set
text_test_oov = torch.sum(torch.where(text_test_pad > text_train_vocab_len, 1, 0), axis = 1)
headline_test_oov = torch.sum(torch.where(headline_test_pad > hl_train_vocab_len, 1, 0), axis = 1)

#### Dataset without OOV - set UNK index to 3

In [52]:
# training data: no 3
text_train_no = torch.where(text_train_pad > text_train_vocab_len, 3, text_train_pad)
headline_train_no = torch.where(headline_train_pad > hl_train_vocab_len, 3, headline_train_pad)

# dev data
text_dev_no = torch.where(text_dev_pad > text_train_vocab_len, 3, text_dev_pad)
headline_dev_no = torch.where(headline_dev_pad > hl_train_vocab_len, 3, headline_dev_pad)

# test data
text_test_no = torch.where(text_test_pad > text_train_vocab_len, 3, text_test_pad)
headline_test_no = torch.where(headline_test_pad > hl_train_vocab_len, 3, headline_test_pad)

### Form datasets containing data information

In [53]:
## Zip text and headline together for dataloader
traindata_zip = torch.utils.data.TensorDataset(text_train_pad, headline_train_pad, 
                                               text_train_padmask, headline_train_padmask, 
                                               text_train_lengths, headline_train_lengths,
                                               text_train_oov, headline_train_oov,
                                               text_train_no, headline_train_no)

devdata_zip = torch.utils.data.TensorDataset(text_dev_pad, headline_dev_pad,
                                             text_dev_padmask, headline_dev_padmask, 
                                             text_dev_lengths, headline_dev_lengths,
                                             text_dev_oov, headline_dev_oov,
                                             text_dev_no, headline_dev_no)

testdata_zip = torch.utils.data.TensorDataset(text_test_pad, headline_test_pad,
                                              text_test_padmask, headline_test_padmask, 
                                              text_test_lengths, headline_test_lengths,
                                              text_test_oov, headline_test_oov,
                                              text_test_no, headline_test_no)

### Save datasets and other metrics

In [54]:
## Save datasets
# train data
torch.save(traindata_zip, './Dataset3/traindata_zip.pt')

# dev data
torch.save(devdata_zip, './Dataset3/devdata_zip.pt')

# test data
torch.save(testdata_zip, './Dataset3/testdata_zip.pt')

In [55]:
# load data by
# temp = torch.load('./Dataset3/traindata_zip.pt')

In [56]:
## Save vocabulary
# train vocab
torch.save(text_vocabulary, './Dataset3/text_vocabulary.pt')
torch.save(headline_vocabulary, './Dataset3/headline_vocabulary.pt')

#### Batch data

In [57]:
## Set batch size and split data after padding to batches
def batch_dataloader(data, Batch_size):
    data_dataloader = torch.utils.data.DataLoader(data, batch_size=Batch_size, shuffle=False, num_workers=0)
    
    return data_dataloader

In [58]:
## Training data batching
trainloader = batch_dataloader(traindata_zip, 100)

devloader = batch_dataloader(devdata_zip, 20)

testloader = batch_dataloader(testdata_zip, 20)

In [59]:
len(text_vocabulary.index2word.keys())

7239